## Primeros Pasos con el Framework de Agentes del SDK de Semantic Kernel

#### Instalando utilidades y dependencias

In [ ]:
#%pip install semantic-kernel==1.28.1

In [ ]:
#%pip install python-dotenv

#### Creando el Kernel y el Cliente AzureChatCompletion

In [ ]:
# --- Importaciones Necesarias ---
from semantic_kernel import Kernel # El orquestador principal.
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion # El conector para Azure OpenAI.
from semantic_kernel.functions import KernelArguments # Para pasar argumentos a las funciones.
import os # Para leer variables de entorno.
import asyncio # Para operaciones asíncronas.
import time # Utilidad de tiempo.
from dotenv import load_dotenv # Para cargar el archivo .env.

# --- Inicialización del Kernel ---
# Se crea la instancia del Kernel, el cerebro de la aplicación.
kernel = Kernel()

# Se carga el archivo .env que contiene las claves y configuraciones.
load_dotenv()

# --- Carga de Variables de Entorno ---
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# (Opcional) Imprime las variables para verificar que se cargaron correctamente.
print(f"API Key: {api_key}")
print(f"Deployment Name: {deployment_name}")
print(f"Endpoint: {endpoint}")

# --- Conexión del Servicio de IA al Kernel ---
# Se le da un alias o ID a nuestro servicio.
service_id = "service1"
# Se añade el servicio de chat de Azure al Kernel para que pueda usar el modelo de IA.
kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key=api_key,
                        deployment_name=deployment_name,
                        endpoint = endpoint
    )
)

#### Creando el Agente con el Cliente AzureChatCompletion

In [2]:
# Importa la clase principal para crear un agente de chat.
from semantic_kernel.agents import ChatCompletionAgent
# Importa clases para configurar cómo el agente ejecuta las funciones (herramientas).
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

# --- Configuración del Comportamiento del Agente ---
# Se preparan los argumentos de ejecución para el agente.
arguments = KernelArguments(
        settings=PromptExecutionSettings(
            # Esta es la configuración CLAVE para el comportamiento "agente".
            # 'FunctionChoiceBehavior.Auto()' le dice al modelo que él mismo decida
            # cuándo debe llamar a una función (herramienta) y que el Kernel
            # la ejecute automáticamente. Esto habilita el "function calling" automático.
            function_choice_behavior=FunctionChoiceBehavior.Auto(),
        )
    )

# --- Creación del Agente ---
# Se crea la instancia del agente de chat.
agent = ChatCompletionAgent(
  kernel=kernel, # Se le pasa el Kernel para que tenga acceso a los servicios de IA y plugins.
  name="ChatCompletionAgent", # Un nombre para identificar al agente.
  instructions="Eres un asistente de IA muy útil", # Instrucciones base sobre cómo debe comportarse.
  arguments=arguments # Se le aplican las configuraciones de ejecución que definimos antes.
)

#### Enviando un mensaje simple a nuestro agente

In [ ]:
# Se llama al agente con un único mensaje. Esto es una conversación de un solo turno, sin memoria.
response = await agent.get_response(messages="Hola, ¿cómo estás?")
# Se imprime la respuesta del agente.
print(response)

#### Viendo en Acción el Historial de Chat del Agente (Thread)

In [ ]:
# Importa la clase para manejar el historial de una conversación.
from semantic_kernel.agents import ChatHistoryAgentThread

# Se define el 'hilo' o 'thread'. Este objeto almacenará el historial de la conversación.
# Es como abrir una ventana de chat persistente.
thread = ChatHistoryAgentThread()

# Se inicializa una variable para controlar el bucle del chat.
continueChat = True

# Comienza un bucle para chatear interactivamente con el agente.
while continueChat:
    # Pide al usuario que ingrese su mensaje.
    user_input = input("Ingresa tu consulta: ")
    # Si el usuario escribe 'exit', se termina la conversación.
    if user_input.lower() == "exit":
        continueChat = False
        break
    # Se llama al agente, pasándole tanto el nuevo mensaje como el objeto 'thread'.
    # Al pasar el 'thread', el agente tendrá acceso a toda la conversación anterior para mantener el contexto.
    response = await agent.get_response(messages=user_input, thread=thread)
    # Se imprime la respuesta del agente en la consola.
    print(response)